In [1]:
import fiona
import rasterio
import rasterio.mask
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import json
import richdem as rd
from shapely.geometry import box
from shapely.geometry import shape, mapping
import pandas as pd
import numpy as np
from geopandas import clip
import scipy
import scipy.spatial
import datetime
import numba

from pathfinder import *

%matplotlib inline
%load_ext line_profiler

import warnings
warnings.filterwarnings('ignore')

#### Define the workspace to read and write data from

In [2]:
workspace = 'burundi'

#### Define the target coordinate reference system (CRS) - should be in meters

In [3]:
crs = 3395

#### Next, define the path to a polygon of your study area (e.g. from GADM)

In [4]:
boundaries_path = r'C:\GitHub\many-to-many-dijkstra\burundi\gadm36_BDI.gpkg'

In [5]:
boundaries = gpd.read_file(boundaries_path).to_crs(crs)

#### In the next step, we create a raster of the study area with a defined resolution

In [6]:
resolution = 50 # meters (same unit as your crs)

In [7]:
total_bounds = boundaries['geometry'].total_bounds
width = round((total_bounds[3] - total_bounds[1])/resolution)
height = round((total_bounds[2] - total_bounds[0])/resolution)

shape =  height, width
transform = rasterio.transform.from_bounds(*boundaries['geometry'].total_bounds, shape[0], shape[1])
rasterized = rasterize(
    [(shape) for shape in boundaries['geometry']],
    out_shape=(width, height),
    transform=transform,
    all_touched=True,
    dtype=rasterio.uint8)

with rasterio.open(
    workspace + '/' 'raster.tif', 'w',
    driver='GTiff',
    dtype=rasterio.uint8,
    count=1,
    crs = crs,
    width=shape[0],
    height=shape[1],
    transform=transform,
    nodata = 0
) as dst:
    dst.write(rasterized, indexes=1)

In [8]:
raster = rasterio.open(workspace + '/' 'raster.tif')

In [9]:
shape = raster.shape
affine = raster.transform

out_meta = raster.meta

out_meta.update({"driver": "GTiff",
                 "height": raster.height,
                 "width": raster.width,
                 "transform": raster.transform,
                 'compress': 'NONE',
                 'dtype': rasterio.float32,
                 "crs": raster.crs,
                 'nodata': 9999})

#### Create cost layer from roads

In [10]:
roads_path = r'C:\GitHub\many-to-many-dijkstra\burundi\roads.gpkg'
roads = gpd.read_file(roads_path).to_crs(crs)

In [11]:
roads["weight"] = 1
roads.loc[roads["highway"] == "motorway", "weight"] = 1 / 10
roads.loc[roads["highway"] == "trunk", "weight"] = 1 / 9
roads.loc[roads["highway"] == "primary", "weight"] = 1 / 8
roads.loc[roads["highway"] == "secondary", "weight"] = 1 / 7
roads.loc[roads["highway"] == "tertiary", "weight"] = 1 / 6
roads.loc[roads["highway"] == "unclassified", "weight"] = 1 / 5
roads.loc[roads["highway"] == "residential", "weight"] = 1 / 4
roads.loc[roads["highway"] == "service", "weight"] = 1 / 3

roads = roads[roads.weight != 1]

roads = roads.sort_values(by="weight", ascending=False)

roads_for_raster = [(row.geometry, row.weight) for _, row in roads.iterrows()]

roads_raster = rasterize(
        roads_for_raster,
        out_shape=shape,
        fill=999,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

with rasterio.open(workspace + '/' + 'roads_cost.tif', 'w', **out_meta) as dst:
    dst.write(roads_raster, indexes=1)

#### Create cost layer from rivers

In [12]:
rivers_path = r'C:\GitHub\many-to-many-dijkstra\burundi\af_riv_30s.shp'
rivers = gpd.read_file(rivers_path, mask=boundaries).to_crs(crs)

In [13]:
rivers['weight'] = 99

rivers_for_raster = [(row.geometry, row.weight) for _, row in rivers.iterrows()]

rivers_raster = rasterize(
        rivers_for_raster,
        out_shape=shape,
        fill=999,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

with rasterio.open(workspace + '/' + 'rivers_cost.tif', 'w', **out_meta) as dst:
    dst.write(rivers_raster, indexes=1)

#### Create cost layer from power lines

In [14]:
power_path = r'C:\GitHub\many-to-many-dijkstra\burundi\burundigrid.geojson'
power = gpd.read_file(power_path, mask=boundaries).to_crs(crs)

In [15]:
power['weight'] = 0

power_for_raster = [(row.geometry, row.weight) for _, row in power.iterrows()]

power_raster = rasterize(
        power_for_raster,
        out_shape=shape,
        fill=999,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

with rasterio.open(workspace + '/' + 'power_cost.tif', 'w', **out_meta) as dst:
    dst.write(power_raster, indexes=1)

#### Create cost layer from water bodies

In [16]:
waters_path = r'C:\GitHub\many-to-many-dijkstra\burundi\africawaterbody.geojson'
waters = gpd.read_file(waters_path, mask=boundaries).to_crs(crs)

In [17]:
waters['weight'] = 99

waters_for_raster = [(row.geometry, row.weight) for _, row in waters.iterrows()]

waters_raster = rasterize(
        waters_for_raster,
        out_shape=shape,
        fill=999,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

with rasterio.open(workspace + '/' + 'waters_cost.tif', 'w', **out_meta) as dst:
    dst.write(waters_raster, indexes=1)

#### Create cost layer from elevation

In [18]:
dem_path = r'C:\GitHub\many-to-many-dijkstra\burundi\dem_burundi.tif'

In [19]:
dem = rasterio.open(dem_path).read(1)
dem_meta = rasterio.open(dem_path).meta

destination = np.ones((out_meta['height'], out_meta['width'])) * 999

elevation, elevation_affine = reproject(
    source=dem,
    destination=destination,
    src_transform=dem_meta['transform'],
    src_crs=dem_meta['crs'],
    dst_transform=out_meta['transform'],
    dst_crs=out_meta['crs'],
    resampling=Resampling['cubic'])



In [20]:
elevation_array = rd.rdarray(elevation, no_data=-9999)
elevation_array.projection = crs
elevation_array.geotransform = elevation_affine
slope = rd.TerrainAttribute(elevation_array, attrib='slope_degrees')

In [21]:
elevation = np.where(elevation > 2000, 1, elevation)
elevation = np.where(elevation > 1600, 0.8, elevation)
elevation = np.where(elevation > 1200, 0.6, elevation)
elevation = np.where(elevation > 800, 0.4, elevation)
elevation = np.where(elevation > 1, 0.2, elevation)
elevation = np.where(elevation < 0.2, 1, elevation)

#### Create final cost layer

In [22]:
costs_1 = np.minimum(rivers_raster, power_raster) 
costs_2 = np.minimum(roads_raster, waters_raster)
costs = np.minimum (costs_1, costs_2)
costs = np.where(costs >= 999, 1, costs)
costs += elevation

costs = np.where((costs > 99) & (costs < 101), 99, costs)

with rasterio.open(workspace + '/' + 'final_cost.tif', 'w', **out_meta) as dst:
    dst.write(costs, indexes=1)

## Perform dijkstra

In [23]:
targets = rasterio.open(r'C:\GitHub\many-to-many-dijkstra\burundi\hydros_cost.tif').read(1)
targets = np.where(targets==10, 1, 0)

In [24]:
origins = np.where(power_raster==0, 1, 0)

In [ ]:
pathfinder = seek(origins, targets, costs, path_handling='link', debug=False, film=True)

In [ ]:
with rasterio.open(workspace + '/' + 'burundi_dijkstra_path.tif', 'w', **out_meta) as dst:
    dst.write(pathfinder['paths'], indexes=1)

In [ ]:
with rasterio.open(workspace + '/' + 'burundi_dijkstra_distance.tif', 'w', **out_meta) as dst:
    dst.write(pathfinder['distance'], indexes=1)